In [1]:
import sys
sys.path.append("./src")

from utils import *
from qiskit.transpiler import CouplingMap

## VQE

In [3]:
hx_min = 0.10526315789473673
hz_min = 2.0
backend = FakeQuitoV2()
H, pauli_strings = build_sparse_hamiltonian(hx = hx_min, hz= hz_min, backend=backend, J=-1, return_paulis=True)
best_params, opt_error = find_best_initial_params_counts(H, pauli_strings, backend)
groups = group_paulis(pauli_strings)

measurement_bases = []
for i, group in enumerate(groups):
    measurement_bases.append(determine_measurement_basis(group))

circuit = build_hva_layers(best_params, backend)
circuits = apply_measurement_bases(circuit, measurement_bases)

counts = []
#simulator = AerSimulator()
simulator = AerSimulator.from_backend(backend)
transpiled_circuits = transpile(circuits, backend=backend, optimization_level=3)
for circuit in transpiled_circuits:
    result = simulator.run(circuit,shots=4*1024).result()
    counts.append(result.get_counts())
    
pauli_expectations = get_pauli_expectation_dict(groups, counts)

energy = get_energy(H, pauli_expectations)
    
exact_energy = compute_exact_ground_energy(H)
rel_error = abs(exact_energy - energy) / abs(exact_energy)
print(f"Relativer Fehler: {rel_error*100} %")

# Optimierung mit COBYLA oder Nelder-Mead (robust bei Noisy Cost Functions):
result = minimize(vqe_cost, best_params,args=(backend,simulator,1,measurement_bases,groups,H,exact_energy), method="COBYLA", options={'maxiter': 50})

print(f"Optimierte Parameter: {result.x}")
print(f"Minimaler relativer Fehler: {result.fun*100}%")

Relativer Fehler: 12.506690993773068 %
Optimierte Parameter: [-8.40283979e-05  1.57076829e+00 -1.57084273e+00]
Minimaler relativer Fehler: 28.678108564628584%


## Subspace VQE

In [6]:
backend = FakeQuitoV2()
hx_min = 0.10526315789473673
hz_min = 2.0
opt_params = [-6.29010556e-05, -1.60006899e+00,  1.57076956e+00]

H, pauli_strings = build_sparse_hamiltonian(hx = hx_min, hz= hz_min, backend=backend, J=-1, return_paulis=True)
circuit = build_hva_layers(opt_params, backend)
pauli_strings = [p.to_label() for p in H.paulis]
unique_paulis = unique_paulis_up_to_N(pauli_strings, N=1)


H_red_observables = generate_all_transformed_hamiltonians(H, unique_paulis)
S_red_observables = generate_all_pauli_products(unique_paulis)

all_pauli_strings = []
for key in H_red_observables.keys():
    paulis1 = [str(p) for p in H_red_observables[key].paulis]
    all_pauli_strings.extend(paulis1)
    
    paulis2 = [str(p) for p in S_red_observables[key].paulis]
    all_pauli_strings.extend(paulis2)

all_paulis_unique = list(set(all_pauli_strings))
all_groups = group_paulis(all_paulis_unique)
all_measurement_bases = []

for i, group in enumerate(all_groups):
    basis = determine_measurement_basis(group)
    all_measurement_bases.append(basis)
    
circuits = apply_measurement_bases(circuit, all_measurement_bases)

counts = []
#simulator = AerSimulator()
simulator = AerSimulator.from_backend(backend)
transpiled_circuits = transpile(circuits, backend=backend, optimization_level=3)
#transpiled_circuits = circuits
for circuit in transpiled_circuits:
    result = simulator.run(circuit,shots=1024).result()
    counts.append(result.get_counts())
    
pauli_expectations = get_pauli_expectation_dict(all_groups, counts)
pauli_list = list(unique_paulis)
n = len(pauli_list)
H_red = np.zeros((n, n), dtype=complex)
S_red = np.zeros((n, n), dtype=complex)

for (P_i, P_k), Observable in H_red_observables.items():
    i = pauli_list.index(P_i)
    j = pauli_list.index(P_k)
    H_red[i, j] = get_energy(Observable, pauli_expectations)
    S_red[i, j] = get_energy(S_red_observables[(P_i, P_k)], pauli_expectations)


#Eigenwerte mit Rauschen sind zum Teil negativ! --> Eigenwertberechnung so

eigvals_S, eigvecs_S = np.linalg.eigh(np.real(S_red))
mask = eigvals_S > 1e-1
V = eigvecs_S[:, mask]

S_proj = V.T.conj() @ S_red @ V
H_proj = V.T.conj() @ H_red @ V

S_inv_sqrt = fractional_matrix_power(S_proj, -0.5)
H_eff = S_inv_sqrt @ H_proj @ S_inv_sqrt

eigvals_proj, eigvecs_proj = np.linalg.eigh(H_eff)
ground_energy = eigvals_proj[0]
ground_state = eigvecs_proj[:, 0]
   
exact_energy = compute_exact_ground_energy(H)
rel_error = abs(exact_energy - ground_energy) / abs(exact_energy)

print(f"Grundzustandsenergie: {ground_energy}")
print(f"Relativer Fehler: {rel_error*100} %")

Grundzustandsenergie: -14.185140696170809
Relativer Fehler: 1.2643372211559456 %


In [4]:
# Lineare Kopplung: 0–1–2–3
coupling = CouplingMap(couplinglist=[(0, 1), (1, 2), (2, 3)])
backend = GenericBackendV2(num_qubits=4, coupling_map=coupling)

num_qubits, coupling_map = get_backend_info(backend)
plot_gate_map(backend)

hx = 1
hz = 1
H_sparse = build_sparse_hamiltonian(hx, hz, backend, J=-1)

best_guess, energy_exact = find_best_initial_params(H_sparse, backend)

opt_params, min_error = optimize_energy(best_guess, H_sparse, backend, energy_exact)

print(f"Initial Error        : {compute_error(best_guess, H_sparse, backend, energy_exact, num_layers=1)}")
print("Exakte Energie       : ", energy_exact)
print("Startparameter       :",best_guess)
print("Optimierte Parameter :", opt_params)
print("Minimaler Fehler     :", min_error)

Initial Error        : 0.10484684299231518
Exakte Energie       :  -7.819890870294844
Startparameter       : [ 0.         -1.57079633  1.57079633]
Optimierte Parameter : [-2.13354542e-05 -1.96813607e+00  1.57075369e+00]
Minimaler Fehler     : 0.004204534323267825
